# Lab 7: PID Control

In previous lab sheets we have developed the following understanding and functionality:
- Familiarised with the Arduino IDE, Sketch files, uploading to the 3Pi+, and utilising the Serial interface
- Implemented basic operation of the 3Pi+ motors, and encapsulated this within safe and confident function(s), and utilised a `class`.
- Implemented basic read functionality of the 3 central IR Line sensors facing the work surface, and encapsulated this within function(s).
- Explored the use of `millis()` to approximate task-scheduling on the 3Pi+.
- Developed a bang-bang controller, using logic to control the robot.
- Implement a calibration routine for the ground sensor.
- Developed a weighted-measurement of the line sensors, calculated an `error signal`, and used a proportional controller to steer the robot to produce line-following behaviour. 
- Implemented a Finite State Machine to better organise your code, and to efficiently manage the operation of your robot. 
- Completed `interrupt service routine` code to count the number of encoder pulses as the wheels rotate.
- Developed kinematics and odometry to track the robot position over time.  


In this labsheet we will investigate a PID controller.  A PID controller can be used in multiple contexts in the line following challenge:
- to control the speed of the wheel rotation, so that they become a more consistent and reliable subsystem.
- to control the heading of the robot, with respect to line following.
- to control the heading of the robot, with respect to $\theta$ in kinematics, allowing for good straight-line travel.
- to control the rotation of the robot, with respect to $\theta$ in kinematics, in order to make precise turn motions.
- to control the travel velocity of the robot, with respect to the distance from origin in kinematics, to allow the robot to return to home in a controlled manner. 

Whilst this may seem like a long list, the PID approach is quite general and you should find it relatively easy to re-apply the techinque into different contexts.  This is one of the strengths of PID control.  

To work on PID, we will first implement a measure of rotation velocity of the 3Pi+ wheels.  With a PID controller, we will then be able to tell the robot to maintain a fixed velocity for each wheel.  This has the clear advantage that:
- if the wheel meets any resistance, the PWM signal will automatically be increased to compensate and restore the velocity.
- if the robot is effected by a decreasing battery power, the PWM signal will automatically be increased to compensate and restore the velocity.
- even if the motors operate differently, setting a desired velocity for left and right motors will effectively match the motors and allow for good straight line travel.


<hr><br><br><br><br>

# Rotation Velocity Estimation

In previous labsheets you have developed code to:
- for various tasks, calculate a period of elapsed time, in milliseconds and microseconds, using `millis()` and `micros()`.
- developed code to effect encoder counts, and to use encoder counts for kinematics.

In this exercise both of these componenets will be used to estimate the rotation velocity of a wheel.  Recall that:
<p align="center">
$v = d / \Delta t$
</p>

Where $v$ is velocity, $d$ is distance, and $\Delta t$ is the change in time.  

On the 3Pi+ we actually have three reasonable locations to implement an estimation of rotation velocity:
1. Within `loop()`, allowing for some variation of time elapsed $\Delta t$ via `millis()` or `micros()`, and recording the difference in encoder counts as $d$.
2. Within the encoder ISRs.  Here, $d$ would always equal 1, because the ISR activates with every rotation of the wheel.  $\Delta t$ would need to be captured.
3. Within a `Timer` ISR, where the `Timer` would be configured so that $\Delta t$ could be fixed to a known period, and the change in encoder counts would be $d$.

In the following exercise, we will adopt the `loop()` method because it is quick to implement and sufficient to complete the line following challenge.  The other two are left to your curiousity.

## Exercise 1: Rotation Velocity Estimation

**Decompose the problem:** In this exercise, follow the normal development of working with just 1 wheel to begin with, and then update your code for both wheels when it is appropriate.

1. **Hypothesise:** What is a serious limitation of implementing the velocity estimate within the encoder ISR?  This is a tricky question!

2. It is recommended to start a new sketch to keep your program as simple as possible.  Include your code to operate the `encoders`.  Once you get this working and refactor your code, you might want to merge it back in with your main software.
  - Implement some `global variables` to track the change of encoder counts for each iteration of `loop()`.
  - Implement some `global variables` to track the change of time per iteration of `loop()`.
  - Implement a `global variable` to store an estimation of rotation velocity.

3. Implement code within `loop()` to:
  - Capture a change of encoder counts.  Remember to pay attention to your variable `data types`.
  - It is not strictly necessary to convert the change in encoder counts to radians or mm, this is at your discretion.
  - Capture the time elapsed between `loop()` iterations.
  - Divide the change of encoder by the time elapsed.
  - Store the result into your global rotation velocity variable.  Remember to think about a suitable `data type`.
  - **Validate:** Use Serial to view your rotation velocity estimation.  You may see a very rough looking signal when you move the wheel by hand.  
  - **Validate:** Use your motor command functions to see how the signal appears at different motor power settings.

4. Either use `delay()` or `millis()` (within an `if()` statement) to investigate how the frequency of calculation this velocity estimate changes quality of the signal.
  - **Hypothesise:** You may see a very spikey or scattered plot. What might be an underlying cause of this?

5. **Refactor:** Before you move on to the second wheel, consider updating your code to either be contained within a `function` or within a `class`.

6. Consider implementing a `low pass filter` for your velocity estimate signal.  This can be quickly and crudely achieved with the following code extract:
  - **Hypothesise:** What is an immediately obvious disadvantage to a low pass filter as provided below?

```c
// An initialised valued
float lpf = 0;
float alpha = 0.2; // value between 0:1
                   // how much weight to new readings?

// Update low pass filter
lpf = ( lpf * (1 - alpha ) ) + ( new_reading * alpha );

```


<hr><br><br><br><br>

# Getting Started with PID


In terms of our robot performance, PID is interesting because it may offer smoother control over the robot motion.  This will be achieved at two levels within the system:
1. Effective control of the wheels speeds.
2. Effective control of the heading of the robot.

In both cases, it is desirable that our robot does not have `jerk` in the motion - this is when the robot makes sudden and/or large changes in the motion.  For our small robot, rapid changes to wheel speed are likely to cause the wheels to slip on the surface.  From the last labsheet, we have been using the rotation of the wheels to estimate the robots `pose` (position and rotation).  Therefore, if the wheels slip on the surface, there will be encoder counts but the robot will not change it's `pose` in reality.  Within the line following challenge, wheel slip will severely effect the ability for your robot to complete the final objective, `return to home`.  

In the above two use cases (wheel speed and heading), we can imagine that rapid changes to heading are likely to cause a dramatic change of required wheels speeds.  This is because the wheel speed will be responsible for orientating the robot to a new heading.  Therefore, it is desirable for our heading to change in a continuous manner (not sudden), just as it is desirable for wheel speed to change in a continuous manner.  

PID will allow us to utilise a continuous control signal relative to some measurement of the system.  The measurement could come from:
- kinematics, such as theta (rotation) or a point to travel to (x,y)
- the position of the line under the robot
- the speed of rotation of the wheels

A nice example of PID being used for smooth motion control is the following video of an e-puck robot tracking a ball with a low-resolution camera: <a href="https://www.youtube.com/watch?v=Pga71leqf1A">YouTube</a>.  Whilst not documented, it appears at least two PID controllers are being used:
(1) tracking left to right position of ball, (2) tracking a set distance from the ball. 




<p align="center">
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/PID_Overview.png?raw=true">
</p>

The above diagram illustrates the general form of a PID controller.  Outside of the blue-dotted box, we can see that a measurement from the system output, $y(t)$ is combined with a desired measurement, $r(t)$, to form the error signal, $e(t)$:

> $e(t) = r(t) - y(t)$


In a previous labsheet, you will have already implemented a `proportional controller` (P-controller) for line following, one of the three P, I and D elements inside the blue-dotted box.  The `integral` and `derivative` components were not investigated.  

The P-Controller implementation allows us to address many parts of the above diagram:
- the **`measurement`**, $y(t)$, was the output from the weighted-measurement calculation, which estimated the line position under the ground sensor in the range [ -1.0 : +1.0 ].
- the **`demand`**, $r(t)$, was set to 0, because it was desired to have the line positioned central between the ground sensors, which equated to a measurement value of 0.  Sometimes `demand` is also known as `setpoint`.
- the **`error signal`**, $e(t)$, meaning the difference between the measurement and the demand, was therefore $e(t) = 0 - y(t)$.
- the **`feedback signal`**, $u(t)$, was calculated by multiplying the error signal by a maximum motor power.  This had the effect of proportionally scaling motor power with respect to the error.
- therefore, the **`proportional gain`**, $K_{p}$, was set as the maximum motor power.  
- the `process` box above was the motor and wheel subsystem of the 3Pi+ robot.  In receiving the feedback signal (`analogWrite()`), the system process was altered, and so the measurement of the line sensors was effected.

We can make some interesting observations:
- the `demand` and the `measurement` were of the same units, because the PID controller is aiming to minimise the `error signal` to 0.  
- the `feedback signal`, $u(t)$, is of different units.  In the line following implementation, the feedback signal was an 8-bit value [0:255] sent to the Arduino function `analogWrite()` to send power to the motor.
- the PID controller is therefore trying to match $r(t)$ and $y(t)$ by manipulating the feedback signal, $u(t)$.
- the effect of the feedback signal $u(t)$ is to effect physical interaction of the robot with the environment, resulting in the new measurement $y(t)$.

In general terms, we can consider the PID controller as acting to transform the error signal into a feedback signal.  The question for a PID controller designer is:
> How can the error signal be usefully transformed into a feedback signal?

In the above diagram we also see the identities $K_{p}$, $K_{i}$ and $K_{d}$.  These are the system `gains`, numerical constants used to manipulate the error signal.  These are the internal parts of the PID algorithm that require a human designer to `tune` (calibrate), to create the useful transformation of error signal to feedback signal.  

The `proportional` component gain, $K_{p}$, is the easiest to gain to consider.  $K_{p}$ simply multiplies with error signal $e(t)$.  If this were a line on a graph, $K_{p}$ would effect the slope of the line.  Therefore, $K_{p}$ will effectively scale a given value of $e(t)$ to produce the feedback signal $u(t)$.




<hr><br><br><br><br>

## Exercise 2: Implement a P-Controller

Similar to previous exercises when developing new functionality, it is recommended you start with a new sketch to keep your program as simple as possible.  You will need to include your encoder, motor and velocity estimation functionality.

1. **Hypothesise:** When we implemented the `proportional` controller for the line following, we knew that the error signal would be in the range [ -1.0 : +1.0 ].  When we are going to measure and set a demand of rotational velocity, this could be in the range [ -inf : +inf ].  
  - What are the implications for finding an effective value of the `gain` $K_{p}$, and it's relation to `demand` and `measurement`?
  - What other techniques, outside of PID and these labsheets have you heard of that might be applicable to help with this problem?

2. Review and update the `pid.h` source file in the template provided on Github (<a href="https://github.com/paulodowd/EMATM0053_21_22/tree/main/3PI_CodeStub">Github Page</a>, <a href="https://github.com/paulodowd/EMATM0053_21_22/raw/main/3PI_CodeStub/Labsheet_X.zip"> Download Link for Zip file </a>) :
  - Implement variables within the class to store the `p_term`, and the `feedback signal`.
  - Utilise an `initialise()` function to set the class variables to 0.  It is recommended you also use `initialise()` to set the initial value of $K_{p}$ via an arugment passed in.
  - Utilise the `update()` function to implement the P-Controller update.
  - Add function arugments: `float update( float demand, float measurement )`.
  - Implement the `error signal` calculation, $e(t) = r(t) - y(t)$.
  - Implement the `p-term`, of the form $ p = K_{p} * e $
  - Return the `p-term` as the feedback signal.

3. At the top of your main program tab (`global variables`), create a new instance of your PID class for the one motor.  Remember to call the `initialise()` function within `setup()`.

4. Within `loop()`, schedule a call to your PID class `update()` function.  A good value to start with is a 20ms interval.
  - If you have followed the above steps, your `update()` function should return the feedback signal.  Capture this feedback signal within `loop()` by assigning it to a variable.
  - When you call `update()`, you will need to add the `demand` and the `measurement`.  
    - Use the velocity estimate of your wheel as the measurement.
    - Set the `demand` to an arbitrary but sensible value.
    - Set gain $K_{p}$ to 10 - this means the error signal will be scaled by a factor of 10.
  - **Validate:** Before you activate the motors, validate:
    - your velocity esimate is functioning properly.
    - your demand appears on a plot.
    - your demand is a sensible value relative to the measurement.
    - your error signal is functioning properly.
    - your feedback signal would cause your wheel to rotate in the right direction.
      - if the relationship is the wrong way around, you can either invert the form of the error signal calculation, or change the sign of $K_{p}$.
  
5. Add code to your `loop()` to pass the feedback signal from your PID class `update()` function into the command to set the power for your motor.
  - Test your robot with motor power.  It is recommended you lift your robot off the surface for the time being.  Using a cup or the reel of black tape beneath the robot is convenient.
  - **Validate:** Use the Serial Monitor or Plotter to check that the subsequent measurements of rotation velocity are closer to your demand.
    - If the `measument` has gone away from the `demand`, your feedback signal is inverted.
    - If you see a lot of oscillation, your gain $K_{p}$ is too large.
    - If you see very little movement (but perhaps, can hear the motor), your $K_{p}$ gain is too small.

6. Attempt to find a value for the gain $K_{p}$ so that for a given sensible demand of rotation velocity, your motor will be driven with a feedback signal to match the measurement to demand.
  - Start with a "too small" value of $K_{p}$ and increase it incrementally.
  - **Hypothesise:** You will likely notice that for a value of $K_{p}$, the closer your measurement gets to the demand when the robot is operating, the more oscillation you will see.  Why might this be?


<hr><br><br><br><br>

# Steady-State Error and the I-Term

In the last exercise when tuning the `gain` $K_{p}$ for a P-Controller we observed that the closer the measurement got to the demand, the more likely it was that there was undesirable oscillation.  In fact, the most stable performance you could achieve probably looked like the following on a  plot illustration:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/PID_SteadyStateError.png?raw=true">
</p>

The above illustrated peformance has to do with the calculation of the error signal:

$e(t) = r(t) - y(t)$

In a P-Controller, as the demand and measurement draw near, the error signal tends towards zero.  This would mean that the feedback signal also tends towards zero.  In the application of `closed-loop` control of the rotation velocity of the wheel, a zero feedback signal would mean the motor becomes deactivated.  

In the above illustration, the `gain` $K_{p}$ has actually be tuned to create an uneasy equilibrium between a non-zero error signal, and a non-zero feedback signal.  This has the consequence that the system can never reach the demand, and this not reliable.  When $K_{p}$ has been tuned for equillibrium like this, it is unlikely to accept a wide range of demand inputs.  As annotated in the above, the persistent gap between the demand and measurement is known as a `steady-state error`.  

This `steady-state error` will occur for P-Controllers in systems where continuous drive is required to keep the error signal minimised.  An intuitive example is a helicopter instructed to hover at an altitude of 10 meters.  The helicopter will be under the effect of gravity, pulling it back to earth.  From an initial state on the ground, our P-Controller would drive the engine, causing lift off.  However, when the helicopter attained 10m altitude, the feedback signal would be 0, and the engine would switch off.

In contrast, we can consider a system like an inkjet printer, which needs to move the print heads to specific locations along a rail.  In this system it would be desirable that as the carriage approaches the destination, the velocity tends towards zero to stop the carriage at the correct destination. After this, no power is required to maintain the position.

Rotation velocity control of the 3Pi+ wheel is more like the helicopter, because frction will quickly work to slow the wheel when power is removed.  

The `I-term` in a PID controller will work to mitigate `steady-state error`.  The I-term will integrate the error signal over time.  This has the effect that the longer the system is in error, the more the I-term will contribute to the overall output feedback signal.  

<p align="center">
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/PID_ITerm.png?raw=true">
</p>




## Exercise 3: Implement a PI-Controller

<hr><br><br><br><br>

# The D-Term

## Exercise 4: Implement a PID Controller

<hr><br><br><br><br>

# A Heading PID Controller



## Exercise 5: Implement Nested-Control

<hr>
# Recommended Reading

Quantisation Error: <a href="">DSP Guide for Engineers Chapter 3.</a>